# 定数変更

In [1]:
#!sed --help

!sed -i s/"'gna_'  : 16"/"'gna_'  : 20"/ ./elecpy/cell/luorudy/const.py 
!sed -i s/"'gacai_' : 0.01"/"'gacai_' : 0.01\/1.2"/ ./elecpy/cell/luorudy/const.py 
!sed -i s/"'gacao_' : 0.341"/"'gacao_' : 0.341\/1.2"/ ./elecpy/cell/luorudy/const.py 

!sed -i s/"'gkr_'   : 0.031368"/"'gkr_'   : 0.71*0.031368"/ ./elecpy/cell/luorudy/const.py 
!sed -i s/"'gks_'   : 0.3031"/"'gks_'   : 0.71*0.3031"/ ./elecpy/cell/luorudy/const.py 

!sed -i s/"(-dt\/_mtau)"/"(-1.2*dt\/_mtau)"/ ./elecpy/cell/luorudy/kernel.c
!sed -i s/"(-dt\/_htau)"/"(-1.2*dt\/_htau)"/ ./elecpy/cell/luorudy/kernel.c

In [2]:
#!cat elecpy/cell/luorudy/const.py 
!cat elecpy/cell/luorudy/const.py  | grep gna_
!cat elecpy/cell/luorudy/const.py  | grep gkr_
!cat elecpy/cell/luorudy/const.py  | grep gks_

!cat elecpy/cell/luorudy/const.py  | grep gacai_
!cat elecpy/cell/luorudy/const.py  | grep gacao_

!cat elecpy/cell/luorudy/kernel.c | grep "dt/_mtau"
!cat elecpy/cell/luorudy/kernel.c | grep "dt/_htau"

#!cat elecpy/cell/luorudy/const.py  | grep gkr_
#!cat elecpy/cell/luorudy/const.py  | grep gks_

  'gna_'  : 20         , # Max. Conductance of the Na Channel (mS/uF)
  'gkr_'   : 0.71*0.031368*sqrt(4.5/5.4), #0.031368*sqrt(ko_/5.4),
  'gks_'   : 0.71*0.3031,
  'gacai_' : 0.01/1.2      , # Activity coefficient of Ca
  'gacao_' : 0.341/1.2     , # Activity coefficient of Ca
_m = _mss - (_mss-m)*exp(-1.2*dt/_mtau);
_h = _hss - (_hss-h)*exp(-1.2*dt/_htau);


# シミュレーション実行

In [3]:
import copy
import time
import os, sys
import shutil
import json
import numpy as np

import chainer
from chainer import cuda

from matplotlib import animation, rc
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import elecpy.elecpy as elp
from elecpy.util.cmap_bipolar import bipolar
from elecpy.elecpySession import ElecpySession

from IPython.display import HTML

from scipy.signal import argrelmax
from numba.decorators import autojit

%matplotlib inline


In [4]:
path_org = "/mnt/recordings/SimulationResult/20180816-1/"
path_dst = "/mnt/recordings/SimulationResult/20180817-2/"

In [8]:
# load sample parameter
with open (os.path.join(path_org, 'sim_params.json'),'r') as f:
    sim_params = json.load(f)

sim_params['restart'] = {}
sim_params['restart']['count'] = 2165
sim_params['restart']['source'] = path_org
    
# modify parameters
sim_params['time']['end'] = 400
sim_params["stimulation"] = {}
#sim_params["stimulation"]["extracellular"][0]["start"] = 0.
#sim_params["stimulation"]["extracellular"][0]["duration"] = 10.

sim_params['geometory']['height'] = 200
sim_params['geometory']['width'] = 200

sim_params['log']['path'] = path_dst
sim_params['log']['cnt'] = 1000


print json.dumps(sim_params, indent=4)


{
    "cell_type": "luorudy", 
    "log": {
        "path": "/mnt/recordings/SimulationResult/20180817-2/", 
        "cnt": 1000
    }, 
    "stimulation": {}, 
    "time": {
        "end": 400, 
        "udt": 0.001
    }, 
    "geometory": {
        "width": 200, 
        "ds": 0.015, 
        "height": 200
    }, 
    "restart": {
        "count": 2165, 
        "source": "/mnt/recordings/SimulationResult/20180816-1/"
    }
}


In [9]:
g = elp.sim_generator(sim_params)

In [10]:
os.mkdir(sim_params['log']['path'] )

OSError: [Errno 17] File exists: '/mnt/recordings/SimulationResult/20180817-2/'

In [11]:
with open(os.path.join(path_dst, 'sim_params.json'),'w') as f: json.dump(sim_params, f, indent=4)
out = []
while True:
    img = g.next()
    if img is False: break
    out.append(np.copy(img))

elecpy simulation start!


/root/.pyenv/versions/anaconda2-4.3.1/lib/python2.7/site-packages/chainer/cuda.py:84: UserWarning: cuDNN is not enabled.
Please reinstall CuPy after you install cudnn
(see https://docs-cupy.chainer.org/en/stable/install.html#install-cupy-with-cudnn-and-nccl).
  'cuDNN is not enabled.\n'


Stimulation settings ...done
Allocating data... ...done
Initializing data... ...done
Building PDE system ... ...done
Main loop start!
------------------400.0/400mselecpy done
